# Quick Start Tutorial: Feature Management

## Learning Objectives

In this tutorial you will learn:
1. How to view the lineage of a feature
2. How to manage the readiness of a feature
3. How to be informed of the readiness of a feature list
4. How to manage the status of a feature list
5. How FeatureByte deployment guardrails work
6. How to check the feature job status
7. How to manage versioning
8. How to disable deployment

## Set up the prerequisites

Learning Objectives

In this section you will:
* connect to the remote featurebyte server
* import libraries
* learn about catalogs
* activate a pre-built catalog

In [ ]:
import urllib.request

# install featurebyte package and download supporting library
!pip install --no-warn-conflicts featurebyte
urllib.request.urlretrieve("https://raw.githubusercontent.com/featurebyte/featurebyte-hosted-tutorials/main/tutorials/notebooks/prebuilt_catalogs.py", "prebuilt_catalogs.py")

In [1]:
# library imports
import pandas as pd
import numpy as np
import random

# load the featurebyte SDK
import featurebyte as fb

# replace <api_token> with your API token you receieved after registering
fb.register_tutorial_api_token("<api_token>")

# define the database name for this tutorial
TUTORIAL_DATABASE = "TUTORIAL_DATASETS"

13:54:27 | INFO     | Using configuration file at: /Users/smillet/.featurebyte/config.yaml
13:54:27 | WARNING  | No valid profile specified. Update config file or specify valid profile name with "use_profile".
13:54:27 | INFO     | Using profile: tutorial
13:54:27 | INFO     | Using configuration file at: /Users/smillet/.featurebyte/config.yaml
13:54:27 | INFO     | Active profile: tutorial (https://tutorials.featurebyte.com/api/v1)
13:54:28 | WARNING  | Remote SDK version (0.3.0.dev136) is different from local (0.4.1). Update local SDK to avoid unexpected behavior.
13:54:28 | INFO     | No catalog activated.
13:54:28 | INFO     | 2 feature lists, 9 features deployed


### Create a pre-built catalog for this tutorial, with the data, metadata, and features already set up

Note that creating a pre-built catalog is not a step you will do in real-life. This is a function specific to this quick-start tutorial to quickly skip over many of the preparatory steps and get you to a point where you can materialize features.

In a real-life project you would do data modeling, declaring the tables, entities, and the associated metadata. This would not be a frequent task, but forms the basis for best-practice feature engineering.

In [2]:
# get the functions to create a pre-built catalog
from prebuilt_catalogs import *

# create a new catalog for this tutorial
catalog = create_tutorial_catalog(PrebuiltCatalog.QuickStartFeatureManagement)

Cleaning up existing tutorial catalogs
Building a quick start catalog for feature management named [quick start feature management 20230726:1354]
Creating new catalog
Catalog created


13:54:43 | INFO     | Catalog activated: quick start feature management 20230726:1354


Registering the source tables
Registering the entities
Tagging the entities to columns in the data tables
Populating the feature store with example features
Done! |████████████████████████████████████████| 100% in 6.7s (0.15%/s)         
Done! |████████████████████████████████████████| 100% in 6.5s (0.16%/s)         
Done! |████████████████████████████████████████| 100% in 3.3s (0.31%/s)         
Loading Feature(s) |████████████████████████████████████████| 1/1 [100%] in 0.5s
Done! |████████████████████████████████████████| 100% in 6.5s (0.16%/s)         
Done! |████████████████████████████████████████| 100% in 9.7s (0.10%/s)         
Loading Feature(s) |████████████████████████████████████████| 1/1 [100%] in 0.5s
Done! |████████████████████████████████████████| 100% in 6.5s (0.15%/s)         
Loading Feature(s) |████████████████████████████████████████| 1/1 [100%] in 0.5s
Done! |████████████████████████████████████████| 100% in 6.5s (0.16%/s)         
Done! |██████████████████████████

## Manage Feature Readiness

Learning Objectives

In this section you will learn:
* how to change the readiness of a feature
* the meaning of each readiness value

### Feature readiness

To help differentiate features that are in the prototype stage and features that are ready for production, a feature version can have one of four readiness levels:

PRODUCTION_READY: ready for deployment in production environments.<br>
PUBLIC_DRAFT: shared for feedback purposes.<br>
DRAFT: in the prototype stage.<br>
DEPRECATED`: not advised for use in either training or prediction.

In [3]:
# list the features in the catalog - note the readiness of each
catalog.list_features()

,id,name,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
0,64c15e674ffda3c15e4bed04,InvoiceUniqueProductGroups,OBJECT,DEPRECATED,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:57:19.204
1,64c15e674ffda3c15e4bed05,InvoiceUniqueProductGroupCount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:56:58.361
2,64c15e5f4ffda3c15e4bed03,InvoiceDiscountAmount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:56:49.969
3,64c15e574ffda3c15e4bed02,InvoiceItemCount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:56:42.382
4,64c15e504ffda3c15e4bed01,CustomerYearOfBirth,INT,PRODUCTION_READY,True,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:34.053
5,64c15e484ffda3c15e4becfe,CustomerSpend_14d,FLOAT,PRODUCTION_READY,True,[GROCERYINVOICE],[GROCERYINVOICE],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:26.084
6,64c15e404ffda3c15e4becfc,CustomerInventory_24w,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:18.913
7,64c15e374ffda3c15e4becfa,CustomerInventory_28d,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:10.849
8,64c15e304ffda3c15e4becf9,StateMeanLongitude,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-07-26 17:56:02.060
9,64c15e284ffda3c15e4becf8,StateMeanLatitude,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-07-26 17:55:54.516


### Example: Set features to production ready

In [4]:
# change the state features to be production ready
for feature_name in catalog.list_features().name:
    feature = catalog.get_feature(feature_name)

    # does the feature name contain the word "state"?
    if "State" in feature.name:
        feature.update_readiness("PRODUCTION_READY")

In [5]:
# list the features in the catalog - note the readiness of each
catalog.list_features()

,id,name,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
0,64c15e674ffda3c15e4bed04,InvoiceUniqueProductGroups,OBJECT,DEPRECATED,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:57:19.204
1,64c15e674ffda3c15e4bed05,InvoiceUniqueProductGroupCount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:56:58.361
2,64c15e5f4ffda3c15e4bed03,InvoiceDiscountAmount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:56:49.969
3,64c15e574ffda3c15e4bed02,InvoiceItemCount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:56:42.382
4,64c15e504ffda3c15e4bed01,CustomerYearOfBirth,INT,PRODUCTION_READY,True,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:34.053
5,64c15e484ffda3c15e4becfe,CustomerSpend_14d,FLOAT,PRODUCTION_READY,True,[GROCERYINVOICE],[GROCERYINVOICE],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:26.084
6,64c15e404ffda3c15e4becfc,CustomerInventory_24w,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:18.913
7,64c15e374ffda3c15e4becfa,CustomerInventory_28d,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:10.849
8,64c15e304ffda3c15e4becf9,StateMeanLongitude,FLOAT,PRODUCTION_READY,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-07-26 17:56:02.060
9,64c15e284ffda3c15e4becf8,StateMeanLatitude,FLOAT,PRODUCTION_READY,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-07-26 17:55:54.516


## List Unsaved Features

Features that have not been saved will not persist once you close your Notebook. It is helpful to check that you have saved each feature that you wished to keep.

In [6]:
# create a feature without saving it
grocery_items_table = catalog.get_table("INVOICEITEMS")
grocery_items_view = grocery_items_table.get_view()
invoice_unique_product_ids = grocery_items_view.groupby(
    "GroceryInvoiceGuid", category="GroceryProductGuid"
).aggregate(None, method=fb.AggFunc.COUNT, feature_name="InvoiceUniqueProductIds")
invoice_unique_product_count = invoice_unique_product_ids.cd.unique_count()

In [7]:
# list unsaved features
fb.list_unsaved_features()

# note that the feature we just created hasn't been named, so cannot be saved

,object_id,variable_name,name,catalog,active_catalog
0,64c15ec44ffda3c15e4bed0f,invoice_unique_product_ids,InvoiceUniqueProductIds,quick start feature management 20230726:1354,True
1,64c15ec44ffda3c15e4bed10,invoice_unique_product_count,None,quick start feature management 20230726:1354,True


## Manage Feature List Status

Learning Objectives

In this section you will learn:
* how to change the status of a feature list
* the meaning of each status value
* how to deploy a feature list

### Feature list status

Feature lists can be assigned one of five status levels to differentiate between experimental feature lists and those suitable for deployment or already deployed.

- DEPLOYED: Assigned to feature list with at least one deployed version.
- TEMPLATE: For feature lists as reference templates or safe starting points.
- PUBLIC_DRAFT: For feature lists shared for feedback purposes.
- DRAFT: For feature lists in the prototype stage.
- DEPRECATED: For outdated or unnecessary feature lists.

In [8]:
# list the feature lists in the catalog - note the status of each
# Note the readiness fraction which represents the proportion of features that are production ready
# Note the online fraction which represents the proportion of features that are being used in production
catalog.list_feature_lists()

,id,name,num_feature,status,deployed,readiness_frac,online_frac,tables,entities,primary_entities,created_at
0,64c15e7c4ffda3c15e4bed0d,InvoiceFeatureList,3,DRAFT,False,0.0,0.0,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[groceryinvoice],[groceryinvoice],2023-07-26 17:57:19.597
1,64c15e764ffda3c15e4bed09,CustomerFeatureList,4,DEPLOYED,True,1.0,1.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[grocerycustomer],[grocerycustomer],2023-07-26 17:57:12.112
2,64c15e6e4ffda3c15e4bed06,StateFeatureList,5,DRAFT,False,1.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[frenchstate],[frenchstate],2023-07-26 17:57:03.476
3,64c15e084ffda3c15e4becf1,very short feature list,1,DRAFT,False,1.0,0.0,[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-07-26 17:55:21.663


### Example: Make a feature list public

When a feature list is reviewed and ready for sharing with other users, change its status to PUBLIC_DRAFT

In [9]:
# get the state feature list
state_feature_list = catalog.get_feature_list("StateFeatureList")

# update the status to public draft
state_feature_list.update_status("PUBLIC_DRAFT")

Loading Feature(s) |████████████████████████████████████████| 5/5 [100%] in 0.5s


In [10]:
# list the feature lists in the catalog - note the status of each
catalog.list_feature_lists()

,id,name,num_feature,status,deployed,readiness_frac,online_frac,tables,entities,primary_entities,created_at
0,64c15e7c4ffda3c15e4bed0d,InvoiceFeatureList,3,DRAFT,False,0.0,0.0,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[groceryinvoice],[groceryinvoice],2023-07-26 17:57:19.597
1,64c15e764ffda3c15e4bed09,CustomerFeatureList,4,DEPLOYED,True,1.0,1.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[grocerycustomer],[grocerycustomer],2023-07-26 17:57:12.112
2,64c15e6e4ffda3c15e4bed06,StateFeatureList,5,PUBLIC_DRAFT,False,1.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[frenchstate],[frenchstate],2023-07-26 17:57:03.476
3,64c15e084ffda3c15e4becf1,very short feature list,1,DRAFT,False,1.0,0.0,[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-07-26 17:55:21.663


### Example: Deploy a feature list

Deploying a feature list changes its status to published

In [11]:
# deploy the state feature list
deployment = state_feature_list.deploy(make_production_ready=True)
deployment.enable()

Loading Feature(s) |████████████████████████████████████████| 5/5 [100%] in 0.5s
Done! |████████████████████████████████████████| 100% in 8.2s (0.12%/s)         
Done! |████████████████████████████████████████| 100% in 12.9s (0.08%/s)        


In [12]:
# list the feature lists in the catalog - note the status of each
catalog.list_feature_lists()

,id,name,num_feature,status,deployed,readiness_frac,online_frac,tables,entities,primary_entities,created_at
0,64c15e7c4ffda3c15e4bed0d,InvoiceFeatureList,3,DRAFT,False,0.0,0.0,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[groceryinvoice],[groceryinvoice],2023-07-26 17:57:19.597
1,64c15e764ffda3c15e4bed09,CustomerFeatureList,4,DEPLOYED,True,1.0,1.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[grocerycustomer],[grocerycustomer],2023-07-26 17:57:12.112
2,64c15e6e4ffda3c15e4bed06,StateFeatureList,5,DEPLOYED,True,1.0,1.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[frenchstate],[frenchstate],2023-07-26 17:57:03.476
3,64c15e084ffda3c15e4becf1,very short feature list,1,DRAFT,False,1.0,0.0,[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-07-26 17:55:21.663


## Versioning

Learning Objectives

In this section you will learn:
* about feature versions and feature list versions
* how to change the table cleaning operations for a feature
* how to change the feature job settings for a feature
* how to manage the default versions for a feature or a feature list
* how to create new versions of features and feature lists

### Concept: Feature version
A Feature Version enables the reuse of a Feature with varying feature job settings or distinct cleaning operations.

If the availability or freshness of the data source change, new versions of the feature can be generated with a new feature job setting. On the other hand, if changes occur in the data quality of the data sources, new versions of the feature can be created with new cleaning operations that address the new quality issues.

To ensure the seamless inference of Machine Learning tasks that depend on the feature, old versions of the feature can still be served without any disruption.

### Example: Get table cleaning operations for a feature

In [13]:
# get the InvoiceDiscountAmount feature
invoice_discount_amount = catalog.get_feature("InvoiceDiscountAmount")

# list the feature versions
display(invoice_discount_amount.list_versions())

,id,name,version,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at,is_default
0,64c15e5f4ffda3c15e4bed03,InvoiceDiscountAmount,V230726,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:56:49.953,True


In [14]:
# No cleaning has been operated on this feature
invoice_discount_amount.info()["table_cleaning_operation"]

{'this': [], 'default': []}

### Example: Updating table cleaning operations for a feature

In [15]:
# update the data cleaning operations in the InvoiceDiscountAmount feature
new_version = invoice_discount_amount.create_new_version(
    table_cleaning_operations=[
        fb.TableCleaningOperation(
            table_name="INVOICEITEMS",
            column_cleaning_operations=[
                fb.ColumnCleaningOperation(
                    column_name="Discount",
                    cleaning_operations=[
                        fb.MissingValueImputation(imputed_value=0.0),
                        fb.ValueBeyondEndpointImputation(
                            type="less_than", end_point=0, imputed_value=None
                        ),
                    ],
                )
            ],
        )
    ]
)

In [16]:
# list the feature versions
feature_versions = invoice_discount_amount.list_versions()

# sort by created_at ascending
feature_versions.sort_values(by="created_at", ascending=True, inplace=True)

# display only the InvoiceUniqueProductGroups feature - note the new version that has been created
display(feature_versions)

,id,name,version,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at,is_default
1,64c15e5f4ffda3c15e4bed03,InvoiceDiscountAmount,V230726,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:56:49.953,False
0,64c15ee7a20ddc7a8ee9c2e0,InvoiceDiscountAmount,V230726_1,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:59:04.066,True


In [17]:
# Check new cleaning info
new_version.info()["table_cleaning_operation"]["this"]

[{'table_name': 'INVOICEITEMS',
  'column_cleaning_operations': [{'column_name': 'Discount',
    'cleaning_operations': [{'imputed_value': 0.0, 'type': 'missing'},
     {'imputed_value': None, 'type': 'less_than', 'end_point': 0}]}]}]

In [18]:
# Check feature definition file
new_version.definition

'# Generated by SDK version: 0.3.0.dev136\nfrom bson import ObjectId\nfrom featurebyte import ColumnCleaningOperation\nfrom featurebyte import ItemTable\nfrom featurebyte import MissingValueImputation\nfrom featurebyte import ValueBeyondEndpointImputation\n\n\n# item_table name: "INVOICEITEMS", event_table name: "GROCERYINVOICE"\nitem_table = ItemTable.get_by_id(ObjectId("64c15deb4ffda3c15e4bece9"))\nitem_view = item_table.get_view(\n    event_suffix=None,\n    view_mode="manual",\n    drop_column_names=[],\n    column_cleaning_operations=[\n        ColumnCleaningOperation(\n            column_name="Discount",\n            cleaning_operations=[\n                MissingValueImputation(imputed_value=0.0),\n                ValueBeyondEndpointImputation(\n                    type="less_than", end_point=0, imputed_value=None\n                ),\n            ],\n        )\n    ],\n    event_drop_column_names=["record_available_at"],\n    event_column_cleaning_operations=[],\n    event_join_column_names=[\n        "Timestamp",\n        "GroceryInvoiceGuid",\n        "GroceryCustomerGuid",\n        "tz_offset",\n    ],\n)\nfeat = item_view.groupby(\n    by_keys=["GroceryInvoiceGuid"], category=None\n).aggregate(\n    value_column="Discount",\n    method="sum",\n    feature_name="InvoiceDiscountAmount",\n    skip_fill_na=True,\n)\noutput = feat\noutput.save(_id=ObjectId("64c15ee7a20ddc7a8ee9c2e0"))\n'

### Concept: Default feature version
    
The default version of a feature streamlines the process of reusing features by providing the most appropriate version. Additionally, it simplifies the creation of new versions of feature lists.

By default, the feature's version with the highest level of readiness is considered, unless the user overrides this selection. In cases where multiple versions share the highest level of readiness, the most recent version is automatically chosen as the default.

When a feature is accessed without specifying a version ID but only by its name, the default version is automatically retrieved.

### Example: The version we just created should be the default as no other version has a higher readiness and it is the latest version.

In [19]:
new_version.is_default

True

### Example: Get the feature job settings for a feature

Note that changing feature job settings will only affect time-aware features e.g. features created using aggregate_over. It will not affect features based upon simple aggregation.

In [20]:
# get the CustomerInventory_28d feature
customer_inventory_28d_feature = catalog.get_feature("CustomerInventory_28d")

# list the feature versions
display(customer_inventory_28d_feature.list_versions())

,id,name,version,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at,is_default
0,64c15e374ffda3c15e4becfa,CustomerInventory_28d,V230726,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:10.833,True


In [21]:
# Get the feature job settings for CustomerInventory_28d
customer_inventory_28d_feature.info()["table_feature_job_setting"]["this"]

[{'table_name': 'GROCERYINVOICE',
  'feature_job_setting': {'blind_spot': '0s',
   'frequency': '3600s',
   'time_modulo_frequency': '90s'}}]

Note that the table name here is the event table associated with the item table as the timestamp originates from this table.

In [22]:
# show the feature job settings for the grocery invoice table
grocery_invoice_table = catalog.get_table("GROCERYINVOICE")
grocery_invoice_table.default_feature_job_setting

FeatureJobSetting(blind_spot='145', frequency='60m', time_modulo_frequency='90s')

### Example: Change the feature job settings for a feature

Note that changing feature job settings will only affect time-aware features e.g. features created using aggregate_over. It will not affect features based upon simple aggregation.

In [23]:
# update the data cleaning operations for the InvoiceDiscountAmount feature to be more conservative
new_version = customer_inventory_28d_feature.create_new_version(
    table_feature_job_settings=[
        fb.TableFeatureJobSetting(
            table_name="GROCERYINVOICE",
            feature_job_setting=fb.FeatureJobSetting(
                blind_spot="160s",
                frequency="60m",
                time_modulo_frequency="90s",
            ),
        ),
    ]
)

In [24]:
# list the feature versions
feature_versions = customer_inventory_28d_feature.list_versions()

# sort by created_at ascending
feature_versions.sort_values(by="created_at", ascending=True, inplace=True)

# note that the new version is a draft, and that the old version remains production ready
display(feature_versions)

,id,name,version,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at,is_default
1,64c15e374ffda3c15e4becfa,CustomerInventory_28d,V230726,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:10.833,True
0,64c15eefa20ddc7a8ee9c325,CustomerInventory_28d,V230726_1,OBJECT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-07-26 17:59:12.074,False


In [25]:
# Get the feature job settings for the new version
new_version.info()["table_feature_job_setting"]["this"]

[{'table_name': 'GROCERYINVOICE',
  'feature_job_setting': {'blind_spot': '160s',
   'frequency': '3600s',
   'time_modulo_frequency': '90s'}}]

In [26]:
# Check feature definition file
new_version.definition

'# Generated by SDK version: 0.3.0.dev136\nfrom bson import ObjectId\nfrom featurebyte import DimensionTable\nfrom featurebyte import FeatureJobSetting\nfrom featurebyte import ItemTable\n\n\n# item_table name: "INVOICEITEMS", event_table name: "GROCERYINVOICE"\nitem_table = ItemTable.get_by_id(ObjectId("64c15deb4ffda3c15e4bece9"))\nitem_view = item_table.get_view(\n    event_suffix=None,\n    view_mode="manual",\n    drop_column_names=[],\n    column_cleaning_operations=[],\n    event_drop_column_names=["record_available_at"],\n    event_column_cleaning_operations=[],\n    event_join_column_names=[\n        "Timestamp",\n        "GroceryInvoiceGuid",\n        "GroceryCustomerGuid",\n        "tz_offset",\n    ],\n)\n\n# dimension_table name: "GROCERYPRODUCT"\ndimension_table = DimensionTable.get_by_id(ObjectId("64c15dee4ffda3c15e4becea"))\ndimension_view = dimension_table.get_view(\n    view_mode="manual", drop_column_names=[], column_cleaning_operations=[]\n)\njoined_view = item_view.join(\n    dimension_view, on="GroceryProductGuid", how="left", rsuffix=""\n)\ngrouped = joined_view.groupby(\n    by_keys=["GroceryCustomerGuid"], category="ProductGroup"\n).aggregate_over(\n    value_column=None,\n    method="count",\n    windows=["28d"],\n    feature_names=["CustomerInventory_28d"],\n    feature_job_setting=FeatureJobSetting(\n        blind_spot="160s", frequency="3600s", time_modulo_frequency="90s"\n    ),\n    skip_fill_na=True,\n)\nfeat = grouped["CustomerInventory_28d"]\noutput = feat\noutput.save(_id=ObjectId("64c15eefa20ddc7a8ee9c325"))\n'

### Example: Change default feature version mode

The new version of CustomerInventory_28d is not the default as this version is a draft and the prior version is production ready.

In [27]:
new_version.is_default

False

The default can be changed only if the default version mode is set as manual.

In [28]:
# guardrail if default version mode is not MANUAL
# the new version cannot be set as the default
try:
    new_version.as_default_version()
except Exception as ex:
    print(ex)

Cannot set default feature ID when default version mode is not MANUAL.


In [29]:
# downgrade current feature readiness to public draft first
customer_inventory_28d_feature.update_readiness("PUBLIC_DRAFT")

# upgrade new version readiness to public draft, new version becomes default
new_version.update_readiness("PUBLIC_DRAFT")
print(new_version.is_default, customer_inventory_28d_feature.is_default)

True False


In [30]:
# change mode to manual and set the original version as default
customer_inventory_28d_feature.update_default_version_mode("MANUAL")
customer_inventory_28d_feature.as_default_version()
print(customer_inventory_28d_feature.is_default)

True


In [31]:
# change new version as default
new_version.as_default_version()
print(new_version.is_default)

True


In [32]:
# upgrade new version readiness to production ready
new_version.update_readiness("PRODUCTION_READY", ignore_guardrails=True)

### Example: Cannot have more than one production ready version of a feature

In [33]:
# change the readiness of the original version of CustomerInventory_28d to production ready
try:
    customer_inventory_28d_feature.update_readiness("PRODUCTION_READY")
except Exception as ex:
    print("Error changing the readiness of the new version to production ready")
    print(ex)

Error changing the readiness of the new version to production ready
Found another feature version that is already PRODUCTION_READY. Please deprecate the feature "CustomerInventory_28d" with ID 64c15eefa20ddc7a8ee9c325 first before promoting the promoted version as there can only be one feature version that is production ready at any point in time. We are unable to promote the feature with ID 64c15e374ffda3c15e4becfa right now.


### Example: Create version of a Feature List

The Feature List Version allows the use of the latest version of each feature. Upon creation of a new feature list version, the latest default versions of features are employed, unless particular feature versions are specified.

In [34]:
# the current default of the feature list has all feature versions production ready
customer_feature_list = catalog.get_feature_list("CustomerFeatureList")
customer_feature_list.list_features()

Loading Feature(s) |████████████████████████████████████████| 4/4 [100%] in 0.5s


,id,name,version,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at,is_default
0,64c15e504ffda3c15e4bed01,CustomerYearOfBirth,V230726,INT,PRODUCTION_READY,True,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:34.038,True
1,64c15e484ffda3c15e4becfe,CustomerSpend_14d,V230726,FLOAT,PRODUCTION_READY,True,[GROCERYINVOICE],[GROCERYINVOICE],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:26.070,True
2,64c15e404ffda3c15e4becfc,CustomerInventory_24w,V230726,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:18.897,True
3,64c15e374ffda3c15e4becfa,CustomerInventory_28d,V230726,OBJECT,PUBLIC_DRAFT,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:10.833,False


In [35]:
# create new version with the new default of CustomerInventory_28d
new_feature_list_version = customer_feature_list.create_new_version()

Loading Feature(s) |████████████████████████████████████████| 4/4 [100%] in 0.5s


In [36]:
new_feature_list_version.list_features()

,id,name,version,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at,is_default
0,64c15eefa20ddc7a8ee9c325,CustomerInventory_28d,V230726_1,OBJECT,PRODUCTION_READY,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-07-26 17:59:12.074,True
1,64c15e504ffda3c15e4bed01,CustomerYearOfBirth,V230726,INT,PRODUCTION_READY,True,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:34.038,True
2,64c15e484ffda3c15e4becfe,CustomerSpend_14d,V230726,FLOAT,PRODUCTION_READY,True,[GROCERYINVOICE],[GROCERYINVOICE],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:26.070,True
3,64c15e404ffda3c15e4becfc,CustomerInventory_24w,V230726,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:18.897,True


In [37]:
# check readiness
new_feature_list_version.info()["production_ready_fraction"]

{'this': 1.0, 'default': 1.0}

### Default version of a Feature List

The default version of a feature list is the version with the highest fraction of production ready features.

The new version is not the default as its production_ready_fraction is lower than the prior version for CustomerInventory_28d.

In [38]:
# the new version is not the default as it is production_ready_fraction is lower than the current default
new_feature_list_version.is_default

True

## Deleting Drafts

While prototyping, you may create and experiment with many features and feature lists. To avoid feature explosions you should do regular cleanups of unused features and feature lists. Note that you cannot delete features or feature lists that have ever been deployed.

### Example: Deleting a feature

In [39]:
# list all of the features in this catalog
display(catalog.list_features())

# note the feature called "unused experimental feature" - this is a feature that is not being used in production, and was rejected as unhelpful

,id,name,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
0,64c15e674ffda3c15e4bed04,InvoiceUniqueProductGroups,OBJECT,DEPRECATED,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:57:19.204
1,64c15e674ffda3c15e4bed05,InvoiceUniqueProductGroupCount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:56:58.361
2,64c15ee7a20ddc7a8ee9c2e0,InvoiceDiscountAmount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:56:49.969
3,64c15e574ffda3c15e4bed02,InvoiceItemCount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:56:42.382
4,64c15e504ffda3c15e4bed01,CustomerYearOfBirth,INT,PRODUCTION_READY,True,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:34.053
5,64c15e484ffda3c15e4becfe,CustomerSpend_14d,FLOAT,PRODUCTION_READY,True,[GROCERYINVOICE],[GROCERYINVOICE],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:26.084
6,64c15e404ffda3c15e4becfc,CustomerInventory_24w,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:18.913
7,64c15eefa20ddc7a8ee9c325,CustomerInventory_28d,OBJECT,PRODUCTION_READY,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:10.849
8,64c15e304ffda3c15e4becf9,StateMeanLongitude,FLOAT,PRODUCTION_READY,True,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-07-26 17:56:02.060
9,64c15e284ffda3c15e4becf8,StateMeanLatitude,FLOAT,PRODUCTION_READY,True,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-07-26 17:55:54.516


In [40]:
# get the feature
feature_to_delete = catalog.get_feature("unused experimental feature")

# delete the feature
feature_to_delete.delete()

In [41]:
# list all the features and note that the deleted feature no longer appears
display(catalog.list_features())

,id,name,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
0,64c15e674ffda3c15e4bed04,InvoiceUniqueProductGroups,OBJECT,DEPRECATED,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:57:19.204
1,64c15e674ffda3c15e4bed05,InvoiceUniqueProductGroupCount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:56:58.361
2,64c15ee7a20ddc7a8ee9c2e0,InvoiceDiscountAmount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:56:49.969
3,64c15e574ffda3c15e4bed02,InvoiceItemCount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:56:42.382
4,64c15e504ffda3c15e4bed01,CustomerYearOfBirth,INT,PRODUCTION_READY,True,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:34.053
5,64c15e484ffda3c15e4becfe,CustomerSpend_14d,FLOAT,PRODUCTION_READY,True,[GROCERYINVOICE],[GROCERYINVOICE],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:26.084
6,64c15e404ffda3c15e4becfc,CustomerInventory_24w,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:18.913
7,64c15eefa20ddc7a8ee9c325,CustomerInventory_28d,OBJECT,PRODUCTION_READY,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-07-26 17:56:10.849
8,64c15e304ffda3c15e4becf9,StateMeanLongitude,FLOAT,PRODUCTION_READY,True,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-07-26 17:56:02.060
9,64c15e284ffda3c15e4becf8,StateMeanLatitude,FLOAT,PRODUCTION_READY,True,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-07-26 17:55:54.516


### Example: Deleting a feature list

In [42]:
# list all of the feature lists in the catalog
display(catalog.list_feature_lists())

# note the feature list called "very short feature list" - this is a feature list that is not being used in production, and was rejected as unhelpful

,id,name,num_feature,status,deployed,readiness_frac,online_frac,tables,entities,primary_entities,created_at
0,64c15e7c4ffda3c15e4bed0d,InvoiceFeatureList,3,DRAFT,False,0.0,0.0,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[groceryinvoice],[groceryinvoice],2023-07-26 17:57:19.597
1,64c15efcd3f396eb49f62640,CustomerFeatureList,4,DEPLOYED,False,1.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[grocerycustomer],[grocerycustomer],2023-07-26 17:57:12.112
2,64c15e6e4ffda3c15e4bed06,StateFeatureList,5,DEPLOYED,True,1.0,1.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[frenchstate],[frenchstate],2023-07-26 17:57:03.476
3,64c15e084ffda3c15e4becf1,very short feature list,1,DRAFT,False,1.0,0.0,[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-07-26 17:55:21.663


In [43]:
# get the feature list
feature_list_to_delete = catalog.get_feature_list("very short feature list")

# delete the feature
feature_list_to_delete.delete()

Loading Feature(s) |████████████████████████████████████████| 1/1 [100%] in 0.5s


In [44]:
# list all the feature lists and note that the deleted feature list no longer appears
display(catalog.list_feature_lists())

,id,name,num_feature,status,deployed,readiness_frac,online_frac,tables,entities,primary_entities,created_at
0,64c15e7c4ffda3c15e4bed0d,InvoiceFeatureList,3,DRAFT,False,0.0,0.0,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[groceryinvoice],[groceryinvoice],2023-07-26 17:57:19.597
1,64c15efcd3f396eb49f62640,CustomerFeatureList,4,DEPLOYED,False,1.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[grocerycustomer],[grocerycustomer],2023-07-26 17:57:12.112
2,64c15e6e4ffda3c15e4bed06,StateFeatureList,5,DEPLOYED,True,1.0,1.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[frenchstate],[frenchstate],2023-07-26 17:57:03.476


## Deployment Guardrails

FeatureByte has guardrails for deployment, and will prevent you from deploying a feature list that has features that are not production ready.

Learning Objectives

In this section you will learn:
* how to check the readiness of a feature list
* how to deploy a feature list

### Example: Check readiness of a feature list

The Feature List Readiness metric provides information to users about the readiness status of a Feature List. This metric represents the percentage of features that are production ready within the given feature list.

In [45]:
# get the invoice feature list
invoice_feature_list = catalog.get_feature_list("InvoiceFeatureList")

# check feature list is ready to be deployed
invoice_feature_list.info()["production_ready_fraction"]

Loading Feature(s) |████████████████████████████████████████| 3/3 [100%] in 0.5s


{'this': 0.0, 'default': 0.0}

In [46]:
invoice_feature_list.list_features()

,id,name,version,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at,is_default
0,64c15e674ffda3c15e4bed04,InvoiceUniqueProductGroups,V230726,OBJECT,DEPRECATED,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:57:19.188,True
1,64c15e5f4ffda3c15e4bed03,InvoiceDiscountAmount,V230726,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:56:49.953,False
2,64c15e574ffda3c15e4bed02,InvoiceItemCount,V230726,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-07-26 17:56:42.367,True


### Example: Deploy a feature list when production readiness is not 100%

In [47]:
# deploy the invoice feature list
try:
    bad_deployment = invoice_feature_list.deploy()
    bad_deployment.enable()
except Exception as ex:
    print("Error deploying the invoice feature list")
    print(ex)

Loading Feature(s) |████████████████████████████████████████| 3/3 [100%] in 0.5s
Done! |████████████████████████████████████████| 100% in 3.3s (0.30%/s)         
Working... |⚠︎                                       | 0% in 3.4s (0.00%/s)      
Error deploying the invoice feature list
Traceback (most recent call last):
  File "/opt/venv/lib/python3.8/site-packages/celery/app/trace.py", line 477, in trace_task
    R = retval = fun(*args, **kwargs)
  File "/opt/venv/lib/python3.8/site-packages/celery/app/trace.py", line 760, in __protected_call__
    return self.run(*args, **kwargs)
  File "/opt/venv/lib/python3.8/site-packages/featurebyte/worker/task_executor.py", line 205, in run
    return run_async(
  File "/opt/venv/lib/python3.8/site-packages/featurebyte/worker/task_executor.py", line 85, in run_async
    return future.result()
  File "/usr/local/lib/python3.8/concurrent/futures/_base.py", line 437, in result
    return self.__get_result()
  File "/usr/local/lib/python3.8/concurrent

In [48]:
# show the feature lists - note that the invoice feature list has not been deployed
catalog.list_feature_lists()

,id,name,num_feature,status,deployed,readiness_frac,online_frac,tables,entities,primary_entities,created_at
0,64c15e7c4ffda3c15e4bed0d,InvoiceFeatureList,3,DRAFT,False,0.0,0.0,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[groceryinvoice],[groceryinvoice],2023-07-26 17:57:19.597
1,64c15efcd3f396eb49f62640,CustomerFeatureList,4,DEPLOYED,False,1.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[grocerycustomer],[grocerycustomer],2023-07-26 17:57:12.112
2,64c15e6e4ffda3c15e4bed06,StateFeatureList,5,DEPLOYED,True,1.0,1.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[frenchstate],[frenchstate],2023-07-26 17:57:03.476


## Disable Deployment

Learning Objectives

In this section you will learn:
* how to disable deployments

### Example: Disabling a deployment

In [49]:
# helper function to disable deployment for a specific feature list
def disable_deployment(feature_list_name):
    # list deployments
    deployments = catalog.list_deployments()

    # just the ones matching this feature list name
    deployments = deployments.loc[deployments.feature_list_name == feature_list_name]

    # disable
    for id in deployments.id:
        deployment = catalog.get_deployment_by_id(id)
        deployment.disable()

In [50]:
# disable the deployments
disable_deployment("CustomerFeatureList")
disable_deployment("StateFeatureList")

Done! |████████████████████████████████████████| 100% in 6.5s (0.16%/s)         
Done! |████████████████████████████████████████| 100% in 6.5s (0.16%/s)         


In [51]:
# show the feature lists status
catalog.list_feature_lists()

,id,name,num_feature,status,deployed,readiness_frac,online_frac,tables,entities,primary_entities,created_at
0,64c15e7c4ffda3c15e4bed0d,InvoiceFeatureList,3,DRAFT,False,0.0,0.0,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[groceryinvoice],[groceryinvoice],2023-07-26 17:57:19.597
1,64c15efcd3f396eb49f62640,CustomerFeatureList,4,PUBLIC_DRAFT,False,1.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[grocerycustomer],[grocerycustomer],2023-07-26 17:57:12.112
2,64c15e6e4ffda3c15e4bed06,StateFeatureList,5,PUBLIC_DRAFT,False,1.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[frenchstate],[frenchstate],2023-07-26 17:57:03.476


## Next Steps

Now that you've completed the quick-start feature management tutorial, you can put your knowledge into practice or learn more:<br>
1) Put your knowledge into practice by creating features in the "credit card dataset feature engineering playground" or "healthcare dataset feature engineering playground" catalogs
2) Learn more about feature engineering via the "Deep Dive Feature Engineering" tutorial
3) Learn about data modeling via the "Deep Dive Data Modeling" tutorial